In [ ]:
# Disclaimer:
# This file is part of the undergraduate thesis of Mr. Efthymis Michalis.
# The thesis was developed under the supervision of Assistant Prof. Aggelos
# Pikrakis, in the Department of Informatics, School of ICT, University of
# Piraeus, Greece.

In [ ]:
import sys
sys.path.append('.')
from methods import img2points

from nntool.trainer import Trainer
import matplotlib.pyplot as plt
import time
from skimage import feature
from skimage.transform import resize, rescale
from sklearn.cluster import DBSCAN
import numpy as np
import tensorflow as tf
import os

In [ ]:
HEIGHT_CLUSTER_EPS = 6
CANY_SIGMA = 2

In [ ]:

cluster = DBSCAN(np.sqrt(2)+1e-2, 2)
height_cluster = DBSCAN(HEIGHT_CLUSTER_EPS, 6)

tf.compat.v1.disable_eager_execution()
model = Trainer('../topology.txt', '../weights')
model.SetSession()
model.Initialize_Vars()
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
            'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [ ]:
time0 = time.time()
pred_plates = []

files = os.listdir("../../plates")
n_files = len(files)

for file in files:
    file_path = os.path.join("../../plates", file)
    img = plt.imread(file_path)

    # scale image
    if img.size > 500*1000:
        per = (500*1000/img.size)**(1/2)
        img = resize(img, (int(img.shape[0]*per), int(img.shape[1]*per)))

    # image to grayscale
    img_gray = img[:, :, 0]*1/3 + img[:, :, 1]*1/3 + img[:, :, 2]*1/3
    if np.max(img_gray) > 1:
        img_gray *= 1/255
    # img_gray = gaussian(img_gray, 0.5)

    # 1st pipeline step: edge detection
    # edge image
    cany_img = feature.canny(img_gray, CANY_SIGMA, 0.3, 0.6)

    points = img2points(cany_img, 0)

    # 2nd pipeline step: clustering

    ## DBSCAN
    cluster.fit(points)
    points_labels = cluster.labels_

    unique_points_labels = set(points_labels)
    corners = []
    for un_label in unique_points_labels:
        mask_un_label = np.where(points_labels == un_label)
        class_points = points[mask_un_label]
        corners.append([np.min(class_points[:, 0]), np.min(class_points[:, 1]),
                        np.max(class_points[:, 0]), np.max(class_points[:, 1])
                        ])
    corners = np.array(corners)

    ## height clustering
    height_cluster.fit(corners[:, [0,2]])
    mask = np.where(height_cluster.labels_ != -1)
    corners = corners[mask]

    # 3rd pipeline step: recognition
    label_obj = []
    for i, corner in enumerate(corners):
        corner[[0,1]] += -1
        corner[[2,3]] += 1
        label_obj.append([corner[1]])
        crop_img = img_gray[corner[0]:corner[2], corner[1]:corner[3]]
        crop_img = 1 - crop_img
        crop_img = crop_img > 0.5
        #                 0x       1x          0y       1y
        dx = corner[2] - corner[0]
        dy = corner[3] - corner[1]
        avg = abs(dx-dy)/2
        panding0 = int(avg)+int(avg != int(avg))
        panding1 = int(avg)
        if dx > dy:
            crop_img = np.concatenate((np.zeros((dx, panding0)), crop_img, np.zeros((dx, panding1))), axis=1)
        elif dx < dy:
            crop_img = np.concatenate((np.zeros((panding0, dy)), crop_img, np.zeros((panding1, dy))), axis=0)

        scale_img = rescale(crop_img, (28-4)/crop_img.shape[0], anti_aliasing=True)
        scale_img = np.concatenate((np.zeros((28-4, 2)), scale_img, np.zeros((28-4, 2))), axis=1)
        scale_img = np.concatenate((np.zeros((2, 28)), scale_img, np.zeros((2, 28))), axis=0)

        confidence = model.Layers[-1].eval(feed_dict={
                         model.Layers[0]: scale_img.reshape(1, 28, 28, 1),
                         model.keep_prob: np.ones((model.DroupoutsProbabilitys.shape[0]))
                })[0]
        confidence_argmax = np.argmax(confidence)
        label = labels[confidence_argmax]

        label_obj[-1].append(label)


    pred_plate = ''
    for label in sorted(label_obj, key=lambda chari: chari[0]):
        pred_plate += label[1]
    pred_plates.append(pred_plate)


time1 = time.time()
print('Total time (sec.) for', n_files,'images:',time1 - time0)
print('FPS:', n_files/(time1 - time0))

In [ ]:
for pred_plate, file in zip(pred_plates, files):
    file_path = os.path.join("../../plates", file)
    img = plt.imread(file_path)
    plt.figure()
    plt.imshow(img)
    plt.title('car plate recognition: {}'.format(pred_plate))
